In [18]:
import os
print("Files in Folder:")
print("\t"+"\n\t".join(os.listdir("../lyft_stock")))

Files in Folder:
	investing.txt
	marketwatch.txt
	marketwatch_parse.txt
	motleyfool.txt
	motleyfool_parse.txt
	schaeffers.txt
	schaeffers_parse.txt


In [19]:
# Open a file
DOC_NAME = "marketwatch"

with open(f'../lyft_stock/{DOC_NAME}_parse.txt') as f:
    lines = [l.strip() for l in f.readlines() if len(l.strip()) > 0]
print(f"Read {len(lines)} lines from {DOC_NAME} file")

Read 98 lines from marketwatch file


In [20]:
# Segment documents by entities and interactions
assert "Entities:" in lines, "Error: did not find 'Entities:' header"
assert "Interactions:" in lines, "Error: did not find 'Interactions:' header"
idx = lines.index("Interactions:")
entity_lines = lines[1:idx]
interaction_lines = lines[idx + 1:]
# print(interaction_lines)

In [21]:
# Parse entities
def parse_entities(entity_lines):
    ent_map = dict()
    for line in entity_lines:
        if '[' in line:
            ent, contents = line.split('[')
            contents = [c.strip() for c in contents[:-1].split(',')]
        else:
            ent, contents = line, []
        assert ent.strip() not in ent_map.keys(), f"Error: entity {ent} already exists in entity map"
        ent_map[ent.strip()] = list(set(contents))
    return ent_map
entities = parse_entities(entity_lines)
print(entities)

{'pandemic,': ['pandemic', 'COVID-19 pandemic'], 'uber,': ['Uber', 'companies', 'Uber Technologies inc.', 'company'], 'lyft,': ['companies', 'Lyft', 'Lyft Int.', 'rival Lyft'], 'consistent lane,': [''], 'earnings preview,': ['fourth quarter results'], 'ride-hailing,': ['ride-hailing industry', 'ride-hailing environment', 'ride-hailing'], 'fourth quarter,': ['fourth quarter'], 'omicron surge,': ['COVID-19 omicron surge'], 'two years': [], 'stability': [], 'Analyst data': [], 'rides': [], 'travel': [], 'US ride-hailing volume': [], 'YipitData analysis': [], 'Raymond James & Associates': ['', 'Raymond James analysts'], "Uber's fourth quarter bookings": [], "Lyft's daily active users": [], 'driver supply': [], 'New Year': [], 'incentives': ['incentives'], 'status of food delivery': [], "Uber's Eats division": ['business', 'Eats'], 'analysts': ['analysts'], 'FactSet': ['FactSet'], 'Estimize': ['Estimize'], 'hedge-fund managers': [], 'executives': [], 'Uber stock': ['Uber shares'], 'Lyft sto

In [83]:
# PARSING HELPERS

def split_attributes(chunk):
    # If no attributes
    if "(" not in chunk:
        return chunk, None
    # print("Attributes called on", chunk)
    
    # Otherwise get attributes
    ent, attrs = chunk.split('(')
    ent = ent.strip()
    attrs = [attr.strip() for attr in attrs[:-1].split(',')]
    return ent.strip(), attrs

def split_chunks(ent_string):
    # Typically, chunks are split on commas. However, nested actions need to be regarded as a chunk so they must be recombined
    # E.g. Stef, {Will, Neil [eat]} -> ["Stef", "{Will, Neil [eat]}""] and not ["Stef", "{Will", "Neil [eat]}"]
    
    # Split on commas    
    ent_list = ent_string.split(',')
    
    # For {} or () chunks, recombine them
    #TODO check for correct ordering and to handle nesting
    stack = []
    open_idxs = []
    close_idxs = []
    for i, ent in enumerate(ent_list):
        if '{' in ent or '(' in ent or '}' in ent or ')' in ent:
            for ch in ent:
                if ch in ['(', '[']:
                    stack.append(ch)
                    open_idxs.append(i)
                if ch in [')', ']']:
                    top = stack.pop()
                    assert ch == ')' and top == '(' or ch == ']' and top == '[' , "Error: misordered brackets in line" + str(ent_list)
                    close_idxs.append(i)

    # Make sure # of open and close brackets are equal 
    assert len(open_idxs) == len(close_idxs), "Mismatch brackets in line: " + str(ent_list)
    for i, open_idx in enumerate(open_idxs):
        # If open and close are in same chunk, no need to merge
        if open_idx == close_idxs[i]:
            continue
        ent_list = ent_list[:open_idx] + [",".join(ent_list[open_idx:close_idxs[i] + 1])] + ent_list[close_idxs[i] + 1:]
    return [ent.strip() for ent in ent_list]

# Parse a line in the Subject-Verb-Object format "ent1, ent2,... [action] ent3, ent4...""
def parse_SVO(nodes, edges, line):
    # Get subj ents
    # TODO: Handle case: {status of food delivery [changing]} [affects] Uber
    subj_ents, rest = [l.strip() for l in line.split('[', maxsplit=1)]
    subj_ents = split_chunks(subj_ents) 
    subj_ents = filter(None, subj_ents) # Drop empty entries
    
    # Get action and obj_ents
    action, obj_ents = rest.split(']', maxsplit=1)
    obj_ents = split_chunks(obj_ents)
    obj_ents = filter(None, obj_ents) # Drop empty entries
    
    # Get attributes
    action, action_attrs = split_attributes(action)
    # Action is always appended first
    action_idx = len(nodes)
    nodes.append((action, 'interaction'))
    
    count_idx = len(nodes)
    # Add edges
    for ent in subj_ents:
        ent = ent.strip()
        if ent[0] == '{':
            # Action is always appended first
            new_action_idx = count_idx
            nodes, edges = parse_SVO(nodes, edges, ent.strip()[1:-1])
            count_idx = len(nodes)
            edges.append((new_action_idx, action_idx))
        else:
            ent, attrs = split_attributes(ent)
            ent_idx = count_idx
            nodes.append((ent, 'entity'))
            count_idx += 1
            # Add any attributes
            if attrs:
                for attr in attrs:
                    nodes.append((attr, 'attribute'))
                    edges.append((count_idx, ent_idx))
                    count_idx += 1
            
            # Add entity-action
            edges.append((ent_idx, action_idx))
    
    for ent in obj_ents:
        ent = ent.strip()
        if ent[0] == '{':
            # Action is always appended first
            new_action_idx = count_idx
            nodes, edges = parse_SVO(nodes, edges, ent.strip()[1:-1])
            count_idx = len(nodes)
            edges.append((action_idx, new_action_idx))
        else:
            ent, attrs = split_attributes(ent)
            ent_idx = count_idx
            nodes.append((ent, 'entity'))
            count_idx += 1
            # Add any attributes
            if attrs:
                for attr in attrs:
                    nodes.append((attr, 'attribute'))
                    edges.append((count_idx, ent_idx))
                    count_idx += 1
            # Add entity-action
            edges.append((action_idx, ent_idx))
            
    count = len(nodes)
    if action_attrs:
        for attr in action_attrs:
            nodes.append((attr, 'attribute'))
            edges.append((count, action_idx))
            count += 1
    
    return nodes, edges

# text = "FactSet [alleges] {analysts [expect] {Uber [post] ajusted loss of 30 cents a share}}"
# text = "FactSet [alleges] {Uber [post] ajusted loss of 30 cents a share}"
# text = "Estimize [gathers] estimates (from analysts, hedge-fund managers, executives, others)"
# text = "Uber, Lyft [release (scheduled, next week)] earnings preview"
print(parse_interactions([text]))

([('release', 'interaction'), ('Uber', 'entity'), ('Lyft', 'entity'), ('earnings preview', 'entity'), ('scheduled', 'attribute'), ('next week', 'attribute')], [(1, 0), (2, 0), (0, 3), (4, 0), (5, 0)])


In [84]:
# Parse all interactions
def parse_interactions(interaction_lines):
    nodes, edges = [], []
    for line in interaction_lines[:]:
        if len(line) == 0:
            continue
        # line = interaction_lines[0]
        # print("Processing:", line)
        nodes, edges = parse_SVO(nodes, edges, line)
    return nodes, edges

nodes, edges = parse_interactions(interaction_lines)

In [80]:
from models import *

PARENT_DOC = "lyft-" + DOC_NAME

node_objs = []

for i, (name, type_) in enumerate(nodes):
    if type_ == 'entity':
        node_objs.append(Entity(str(i), name, PARENT_DOC))
    elif type_ == 'interaction':
        node_objs.append(Interaction(str(i), name, PARENT_DOC))
    elif type_ == 'attribute':
        node_objs.append(Attribute(str(i), name,  PARENT_DOC))
    else:
        print("Error, unrecognized type:", type_)

assert len(node_objs) == len(nodes), "Error: mismatched lengths of nodes and nodes objects"

# nodes = dict()
# for src, dest in edges:
#     for node in [src, dest]:
#         key, type_ = node
#         if type_ == 'entity':
#             if (key, type_) in nodes.keys():
#                 nodes[key, type_].raw_count += 1
#             else:
#                 nodes[key, type_] = Entity(key, PARENT_DOC)
#         elif type_ == 'interaction':
#             if (key, type_) in nodes.keys():
#                 nodes[key, type_].raw_count += 1
#             else:
#                 nodes[key, type_] = Interaction(key, PARENT_DOC)
#         elif type_ == 'attribute':
#             if (key, type_) in nodes.keys():
#                 nodes[key, type_].raw_count += 1
#             else:
#                 nodes[key, type_] = Attribute(key, PARENT_DOC)
#         else:
#             print("Error, unrecognized type:", type_)

In [81]:
# Upload to DB
from graph_driver import GraphDBDriver
from models import *

driver = GraphDBDriver(remote=False)
print("Uploading nodes")
driver.upload_nodes(node_objs)
# driver.upload_nodes([node for node in nodes.values()])

print("Adding Edges")
edge_objs = []
for source, dest in edges:
    edge_objs.append(Edge("relation", node_objs[source], node_objs[dest]))
driver.upload_edges(edge_objs)
# print("Cleaning up")
# driver.raw_query("MATCH (n:entity) WHERE n.key=\"entity1\" DETACH DELETE n")
# driver.raw_query("MATCH (n:interaction) WHERE n.key=\"interacted\" DETACH DELETE n")
# print("Deleted nodes")
driver.close()
print("Finished")

Uploading nodes
Uploaded 0
Uploaded 1
Uploaded 2
Uploaded 3
Uploaded 4
Uploaded 5
Uploaded 6
Uploaded 7
Uploaded 8
Uploaded 9
Uploaded 10
Uploaded 11
Uploaded 12
Uploaded 13
Uploaded 14
Uploaded 15
Uploaded 16
Uploaded 17
Uploaded 18
Uploaded 19
Uploaded 20
Uploaded 21
Uploaded 22
Uploaded 23
Uploaded 24
Uploaded 25
Uploaded 26
Uploaded 27
Uploaded 28
Uploaded 29
Uploaded 30
Uploaded 31
Uploaded 32
Uploaded 33
Uploaded 34
Uploaded 35
Uploaded 36
Uploaded 37
Uploaded 38
Uploaded 39
Uploaded 40
Uploaded 41
Uploaded 42
Uploaded 43
Uploaded 44
Uploaded 45
Uploaded 46
Uploaded 47
Uploaded 48
Uploaded 49
Uploaded 50
Uploaded 51
Uploaded 52
Uploaded 53
Uploaded 54
Uploaded 55
Uploaded 56
Uploaded 57
Uploaded 58
Uploaded 59
Uploaded 60
Uploaded 61
Uploaded 62
Uploaded 63
Uploaded 64
Uploaded 65
Uploaded 66
Uploaded 67
Uploaded 68
Uploaded 69
Uploaded 70
Uploaded 71
Uploaded 72
Uploaded 73
Uploaded 74
Uploaded 75
Uploaded 76
Uploaded 77
Uploaded 78
Uploaded 79
Uploaded 80
Uploaded 81
Uploaded 8